In [10]:
import cv2
import os

# Create a directory to store the frames
if not os.path.exists("frames"):
    os.makedirs("frames")

# Read the video
video = cv2.VideoCapture("video.mp4")
success, frame = video.read()
count = 0

while success:
    cv2.imwrite(f"frames/frame{count:04d}.jpg", frame)
    success, frame = video.read()
    count += 1

video.release()
print(f"Extracted {count} frames.")


Extracted 94 frames.


In [11]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

frame_folder = "frames"
processed_folder = "processed_frames"

# Create a directory to store the processed frames
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith(".jpg")])

for frame_file in frame_files:
    reference = cv2.imread(os.path.join(frame_folder, frame_file), cv2.IMREAD_GRAYSCALE)
    M, N = reference.shape
    low_quality = reference * 0.01
    threshold = np.sum(low_quality) / (M * N)
    mse = np.zeros(50)
    sigma = 0
    SR = np.zeros((M, N))
    
    for epoch in range(50):
        sigma += 0.1
        for _ in range(10):
            noisy = sigma * np.random.rand(M, N) + low_quality
            modified = np.zeros_like(noisy)
            for i in range(M):
                for j in range(N):
                    if noisy[i, j] > threshold:
                        modified[i, j] = 255
                    else:
                        modified[i, j] = 0
            SR += modified
        SR /= 10
        error = reference - SR
        mse[epoch] = np.sum(error**2) / (M * N)
        mse[epoch] = np.sqrt(mse[epoch])

        if epoch > 0:
            if mse[epoch - 1] > mse[epoch]:
                best = SR.copy()

    processed_frame_path = os.path.join(processed_folder, frame_file)
    cv2.imwrite(processed_frame_path, best)
    print(f"Processed frame: {frame_file}")

Processed frame: frame0000.jpg
Processed frame: frame0001.jpg
Processed frame: frame0002.jpg
Processed frame: frame0003.jpg
Processed frame: frame0004.jpg
Processed frame: frame0005.jpg


In [8]:
import cv2
import os

processed_folder = "processed_frames"
output_video = "output_video.mp4"
frame_files = sorted([f for f in os.listdir(processed_folder) if f.endswith(".jpg")])

# Read the first frame to get the dimensions
frame = cv2.imread(os.path.join(processed_folder, frame_files[0]))
height, width = frame.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(output_video, fourcc, 30, (width, height), isColor=False)

for frame_file in frame_files:
    frame = cv2.imread(os.path.join(processed_folder, frame_file), cv2.IMREAD_GRAYSCALE)
    video.write(cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))

video.release()
print(f"Processed video saved as {output_video}")


AttributeError: 'NoneType' object has no attribute 'shape'